# Video Processing

In [69]:
import pandas as pd
import cv2
from pathlib import Path
from typing import Tuple
from ivr_utils.ivr_utils import find_participant_files

# Constants
PARTICIPANT_ID = "P48"

local_data_dir = Path.cwd().parent / "data"
server_data_dir = Path(
    "/Volumes/ritd-ag-project-rd00pj-jkang71/UCL_SSID/SSID IVR Study 1"
)
output_dir = server_data_dir / "output"

assert server_data_dir.is_dir(), "Server data directory not found"

ModuleNotFoundError: No module named 'moviepy.editor'

## Find Files for participant

In [57]:
eyetracking_dir = server_data_dir / "Eyetracking"

part_csv_path, part_wmv_path = find_participant_files(PARTICIPANT_ID, eyetracking_dir)

# Import csv file for participant

In [77]:
points = pd.read_csv(part_csv_path, skiprows=lambda x: x < 26)

# find the 'StartMedia' timestamp
row = points[points["SlideEvent"] == "StartMedia"]
timestamp_diff = row["Timestamp"].values[0]

# clean the NaN in columns
points = points.dropna(subset=["Gaze X", "Gaze Y"])

# - timestamp_diff to make the timestamp start from 0
points["Timestamp"] = points["Timestamp"] - timestamp_diff

/var/folders/6t/7h8wn9n92w5f24ml_bkwck9m0000gn/T/ipykernel_50360/782891080.py:1: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  points = pd.read_csv(part_csv_path, skiprows=lambda x: x < 26)


## Convert to MP4 if not already found

In [66]:
part_mp4_path = part_wmv_path.with_suffix(".mp4")
assert part_mp4_path.exists(), "MP4 does not exist, try converting the wmv file."

# TODO: Implement mp4 conversion

## Overlaying

In [74]:
# TODO: Extract into utils function

N_FRAMES_PROC = 30 * 60  # 30 fps * 60 seconds

# read the video
# input_video_path = "input.mp4"
output_overlay_path = local_data_dir / f"{PARTICIPANT_ID}_output_overlay_test.mp4"

video = cv2.VideoCapture(part_mp4_path.as_posix())
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter mp4-mp4v format
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_overlay = cv2.VideoWriter(output_overlay_path, fourcc, fps, (width, height))

# # clean the NaN in columns
# points = points.dropna(subset=["Gaze X", "Gaze Y"])

# # - timestamp_diff to make the timestamp start from 0
# points["Timestamp"] = points["Timestamp"] - timestamp_diff

# loop video frames
current_point_index = 0
for frame_index in range(total_frames):
    if frame_index > N_FRAMES_PROC:
        break

    ret, frame = video.read()
    if not ret:
        break

    # time of the frame in ms
    current_time = frame_index / fps * 1000

    # check if the time has reached the next timestamp in the eyetracking csv
    while (
        current_point_index < len(points) - 1
        and points["Timestamp"].iloc[current_point_index + 1] <= current_time
    ):
        current_point_index += 1  # index=index+1

    # get the coordinates corresponding to the current timestamp
    if current_point_index < len(points):
        row = points.iloc[current_point_index]
        x, y = int(row["Gaze X"]), int(row["Gaze Y"])
        # draw a yellow circle
        cv2.circle(frame, (x, y), 50, (0, 250, 250), -1)

    out_overlay.write(frame)

video.release()
out_overlay.release()
cv2.destroyAllWindows()

## Chopping

In [78]:
N_FRAMES_PROC = 30 * 240  # 30 fps * 240 seconds

# read the video
input_path = part_mp4_path
output_video_path = local_data_dir / f"{PARTICIPANT_ID}_output_chopped_test.mp4"

video = cv2.VideoCapture(input_path)
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# # - timestamp difference
# points["Timestamp"] = points["Timestamp"] - timestamp_diff

# process frames
current_point_index = 0
skip_frames = False

for frame_index in range(total_frames):
    if frame_index > N_FRAMES_PROC:
        break

    ret, frame = video.read()
    if not ret:
        break

    current_time = frame_index / fps * 1000

    # update current_point_index based on time
    while (
        current_point_index < len(points) - 1
        and points["Timestamp"].iloc[current_point_index + 1] <= current_time
    ):
        current_point_index += 1

        # skip frames
        if (
            pd.isna(points["Respondent Annotations active"].iloc[current_point_index])
            or points["Respondent Annotations active"].iloc[current_point_index] == ""
        ):
            skip_frames = True
        else:
            skip_frames = False

    # write frame if not skipping
    if not skip_frames:
        out.write(frame)

# release resources
video.release()
out.release()
cv2.destroyAllWindows()